In [ ]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from sklearn.metrics import confusion_matrix, accuracy_score

# Definisi variabel fuzzy
suhu_luar = ctrl.Antecedent(np.arange(10, 41, 1), 'Suhu Luar')
suhu_dalam = ctrl.Antecedent(np.arange(10, 41, 1), 'Suhu Dalam')
jumlah_orang = ctrl.Antecedent(np.arange(0, 51, 1), 'Jumlah Orang')
suhu_optimal = ctrl.Consequent(np.arange(15, 26, 1), 'Suhu Optimal')

# Fungsi keanggotaan
suhu_luar['Sejuk'] = fuzz.trapmf(suhu_luar.universe, [10, 10, 20, 24])
suhu_luar['Normal'] = fuzz.trimf(suhu_luar.universe, [22, 27.5, 33])
suhu_luar['Panas'] = fuzz.trapmf(suhu_luar.universe, [32, 36, 40, 40])

suhu_dalam['Dingin'] = fuzz.trapmf(suhu_dalam.universe, [10, 10, 18, 24])
suhu_dalam['Normal'] = fuzz.trimf(suhu_dalam.universe, [22, 26.5, 31])
suhu_dalam['Hangat'] = fuzz.trapmf(suhu_dalam.universe, [30, 34, 40, 40])

jumlah_orang['Sedikit'] = fuzz.trapmf(jumlah_orang.universe, [0, 0, 10, 18])
jumlah_orang['Sedang'] = fuzz.trimf(jumlah_orang.universe, [12, 23, 34])
jumlah_orang['Banyak'] = fuzz.trapmf(jumlah_orang.universe, [30, 40, 50, 50])

suhu_optimal['Dingin'] = fuzz.trimf(suhu_optimal.universe, [15, 15.5, 17])
suhu_optimal['Cukup Dingin'] = fuzz.trimf(suhu_optimal.universe, [17, 18, 19])
suhu_optimal['Normal'] = fuzz.trimf(suhu_optimal.universe, [19, 20, 21])
suhu_optimal['Cukup Hangat'] = fuzz.trimf(suhu_optimal.universe, [21, 22, 23])
suhu_optimal['Hangat'] = fuzz.trimf(suhu_optimal.universe, [23, 24.5, 25])

# Rule base
rules = []
kondisi = ['Sejuk', 'Normal', 'Panas']
dalam = ['Dingin', 'Normal', 'Hangat']
orang = ['Sedikit', 'Sedang', 'Banyak']
output_map = {
    ('Sejuk','Dingin','Sedikit'):'Cukup Dingin',
    ('Sejuk','Dingin','Sedang'):'Normal',
    ('Sejuk','Dingin','Banyak'):'Cukup Hangat',
    ('Sejuk','Normal','Sedikit'):'Normal',
    ('Sejuk','Normal','Sedang'):'Cukup Hangat',
    ('Sejuk','Normal','Banyak'):'Hangat',
    ('Sejuk','Hangat','Sedikit'):'Cukup Hangat',
    ('Sejuk','Hangat','Sedang'):'Hangat',
    ('Sejuk','Hangat','Banyak'):'Hangat',
    ('Normal','Dingin','Sedikit'):'Normal',
    ('Normal','Dingin','Sedang'):'Cukup Hangat',
    ('Normal','Dingin','Banyak'):'Hangat',
    ('Normal','Normal','Sedikit'):'Cukup Hangat',
    ('Normal','Normal','Sedang'):'Hangat',
    ('Normal','Normal','Banyak'):'Hangat',
    ('Normal','Hangat','Sedikit'):'Hangat',
    ('Normal','Hangat','Sedang'):'Hangat',
    ('Normal','Hangat','Banyak'):'Hangat',
    ('Panas','Dingin','Sedikit'):'Cukup Dingin',
    ('Panas','Dingin','Sedang'):'Dingin',
    ('Panas','Dingin','Banyak'):'Dingin',
    ('Panas','Normal','Sedikit'):'Cukup Dingin',
    ('Panas','Normal','Sedang'):'Dingin',
    ('Panas','Normal','Banyak'):'Dingin',
    ('Panas','Hangat','Sedikit'):'Dingin',
    ('Panas','Hangat','Sedang'):'Dingin',
    ('Panas','Hangat','Banyak'):'Dingin',
}

for sl in kondisi:
    for sd in dalam:
        for jo in orang:
            out = output_map.get((sl, sd, jo))
            if out:
                rules.append(ctrl.Rule(suhu_luar[sl] & suhu_dalam[sd] & jumlah_orang[jo], suhu_optimal[out]))

# Sistem kontrol dan simulasi
suhu_ctrl = ctrl.ControlSystem(rules)
simulasi = ctrl.ControlSystemSimulation(suhu_ctrl)

# Fungsi fuzzy Mamdani
def fuzzy_mamdani(s_luar, s_dalam, jml_orang):
    simulasi.input['Suhu Luar'] = s_luar
    simulasi.input['Suhu Dalam'] = s_dalam
    simulasi.input['Jumlah Orang'] = jml_orang
    simulasi.compute()
    return simulasi.output['Suhu Optimal']

# Load dataset
df = pd.read_csv("dataset_uji_fuzzy.csv")

# Prediksi menggunakan fuzzy Mamdani
df['prediksi'] = df.apply(lambda row: round(fuzzy_mamdani(row['suhu_luar'], row['suhu_dalam'], row['jumlah_orang']), 1), axis=1)

# Menentukan kolom suhu aktual
if 'suhu_optimal_aktual' in df.columns:
    df['suhu_aktual'] = df['suhu_optimal_aktual']
else:
    potential_columns = ['actual_temp', 'suhu_sebenarnya', 'target', 'actual', 'suhu_target']
    found = False
    for col in potential_columns:
        if col in df.columns:
            df['suhu_aktual'] = df[col]
            found = True
            break
    if not found:
        df['suhu_aktual'] = df['prediksi']

# Klasifikasi suhu aktual dan prediksi
df['aktual_kelas'] = df['suhu_aktual'].apply(lambda x: 0 if x <= 20 else 1)
df['prediksi_kelas'] = df['prediksi'].apply(lambda x: 0 if x <= 20 else 1)

# Evaluasi performa
cm = confusion_matrix(df['aktual_kelas'], df['prediksi_kelas'])
tn, fp, fn, tp = cm.ravel()
akurasi = accuracy_score(df['aktual_kelas'], df['prediksi_kelas']) * 100
sensitivitas = tp / (tp + fn) * 100 if (tp + fn) != 0 else 0

# Output evaluasi
print("\n=== HASIL EVALUASI FUZZY MAMDANI ===")
print(f"Akurasi      : {akurasi:.2f}%")
print(f"Sensitivitas : {sensitivitas:.2f}%")

# Tampilkan semua data sample
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print("\n=== DATA LENGKAP HASIL PREDIKSI ===")
print(df[['suhu_luar', 'suhu_dalam', 'jumlah_orang', 'suhu_aktual', 'prediksi']])

# Simpan hasil prediksi ke file CSV
df.to_csv("hasil_prediksi_mamdani.csv", index=False)
print("\nHasil prediksi telah disimpan ke 'hasil_prediksi_mamdani.csv'")


=== HASIL EVALUASI FUZZY MAMDANI ===
Akurasi      : 75.23%
Sensitivitas : 78.08%

=== DATA LENGKAP HASIL PREDIKSI ===
     suhu_luar  suhu_dalam  jumlah_orang  suhu_aktual  prediksi
0         22.0        23.5            10         19.5      19.6
1         30.0        20.0            15         22.0      20.9
2         17.0        19.5            25         21.0      20.0
3         20.5        25.0            12         20.0      20.0
4         34.5        20.0            30         17.0      16.0
5         36.0        29.0            35         15.5      16.0
6         32.0        22.5            18         22.0      22.8
7         33.0        31.0            40         15.0      16.0
8         28.0        23.0            27         21.5      23.1
9         26.5        24.0            12         22.0      22.0
10        21.5        28.0            16         20.0      21.2
11        25.0        21.0            14         16.5      20.6
12        17.0        26.5            35         